# Predicción de categorías delictivas

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

C:\Users\ADMIN\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Carga de datos

In [2]:
crimes_afluency_ft = pd.read_csv('./data_crimes_afluency_line_type.csv', 
                 sep=',', engine='python', encoding="UTF-8", index_col=False)

In [3]:
crimes_afluency_ft.head(3)

,line,type,weekday,id_crime_category,afluency_distribution,crime_frequency
0,1,Intermedia,1,5,11130.083333,1.0
1,1,Intermedia,1,6,11130.083333,10.0
2,1,Intermedia,1,7,11130.083333,0.0


In [4]:
crimes_afluency_ft.columns.values

array(['line', 'type', 'weekday', 'id_crime_category',
       'afluency_distribution', 'crime_frequency'], dtype=object)

## Preprocesamiento

Establecemos columnas categóricas a codificar con OneHotEncoder, y por otro lado las numéricas que serán sometidas a reescalamiento

In [5]:
# Columnas categóricas a codificar con OneHotEncoder
categorical_cols = ['line', 'type', 'weekday', 'id_crime_category']

# Columnas numéricas a escalar con StandardScaler
numeric_cols = ['afluency_distribution']

Definimos pipeline con el preprocesamiento deseado

In [6]:
# Definir el preprocesamiento con ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_cols),
        ('scaler', StandardScaler(), numeric_cols)
    ])

Filtramos dataset para categoría delictiva específica
- 5: ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA
- 6: DELITO DE BAJO IMPACTO
- 7: ROBO DE VEHÍCULO CON Y SIN VIOLENCIA
- 11: ROBO A NEGOCIO CON VIOLENCIA
- 13: ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA

In [7]:
crimes_afluency_ft = crimes_afluency_ft[crimes_afluency_ft['id_crime_category'] == 6]

## Entrenamiento con 80% de los datos

In [8]:
target = 'crime_frequency'

In [9]:
X = crimes_afluency_ft.drop([target], axis = 1)
y = crimes_afluency_ft[target]

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)

## GridSearch

In [11]:
import time
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor, Ridge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from tabulate import tabulate

In [12]:
# Definir el preprocesamiento con ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_cols),
        ('scaler', StandardScaler(), numeric_cols)
    ])

In [13]:
# Definir los modelos con sus hiperparámetros
models = [
    ('MLPRegressor', MLPRegressor(), {
        #'regressor__hidden_layer_sizes': [(50, 50), (100, 100)],
        'regressor__activation': ['relu', 'tanh'],
        'regressor__alpha': [0.0001],
        'regressor__learning_rate_init': [0.0001],
        'regressor__max_iter': [500],
        'regressor__random_state': [1],
        'regressor__verbose': [True]
    }),
    ('SGDRegressor', SGDRegressor(), {
        'regressor__loss': ['squared_loss', 'huber'],
        'regressor__penalty': ['l2', 'l1'],
        'regressor__alpha': [0.0001, 0.001],
        'regressor__learning_rate': ['constant', 'adaptive'],
        'regressor__max_iter': [1000, 2000],
        'regressor__random_state': [1]
    }),
    ('GradientBoostingRegressor', GradientBoostingRegressor(), {
        'regressor__loss': ['ls', 'lad'],
        'regressor__learning_rate': [0.01, 0.1],
        'regressor__n_estimators': [100, 200],
        'regressor__max_depth': [3, 5],
        'regressor__random_state': [1]
    }),
    ('RandomForestRegressor', RandomForestRegressor(), {
        'regressor__n_estimators': [100, 200],
        'regressor__max_depth': [None, 5],
        'regressor__min_samples_split': [2, 5],
        'regressor__random_state': [1]
    }),
    ('LinearRegression', LinearRegression(), {}),
    ('Ridge', Ridge(), {
        'regressor__alpha': [0.1, 1.0],
        'regressor__solver': ['auto', 'svd', 'cholesky']
    }),
    ('PolynomialRegression', Pipeline([
        ('poly', PolynomialFeatures()),
        ('linear', LinearRegression())
    ]), {
        'regressor__poly__degree': [2, 3, 4],
        'regressor__linear__normalize': [True, False]
    }),
    ('SVR', SVR(), {
        'regressor__kernel': ['linear', 'rbf'],
        'regressor__C': [1.0, 10.0],
        'regressor__epsilon': [0.1, 0.01]
    })
]

In [14]:
# Definir los scorers
scoring = {
    'MAE': make_scorer(mean_absolute_error),
    'MSE': make_scorer(mean_squared_error),
    'RMSE': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))),
    'R2': make_scorer(r2_score)
}

In [ ]:
# Lista para almacenar los resultados de cada modelo
results = []

# Realizar la grid search para cada modelo
for name, model, params in models:
    start_time = time.time()

    # Definir el pipeline con el preprocesador y el modelo actual
    model_pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('regressor', model)
        ])
    
    # Realizar la grid search con validación cruzada y obtener los mejores parámetros
    grid_search = GridSearchCV(model_pipeline, params, scoring=scoring, cv=5, refit='RMSE', verbose=3)
    grid_search.fit(X_train, y_train)

    # Obtener los mejores hiperparámetros y métricas de evaluación
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_mae = grid_search.best_score_
    best_mse = grid_search.cv_results_['mean_test_MSE'][grid_search.best_index_]
    best_rmse = grid_search.cv_results_['mean_test_RMSE'][grid_search.best_index_]
    best_r2 = grid_search.cv_results_['mean_test_R2'][grid_search.best_index_]
    train_time = grid_search.refit_time_
    
    # Calcular el tiempo de predicción en el conjunto de validación
    #start_time = time.time()
    #y_pred = best_model.predict(X_validation)
    #end_time = time.time()
    #val_time = end_time - start_time
    
    # Agregar los resultados a la lista
    results.append([name, best_params, best_mae, best_mse, best_rmse, best_r2, train_time])

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Iteration 1, loss = 11.17374439
Iteration 2, loss = 9.83032337
Iteration 3, loss = 8.63791446
Iteration 4, loss = 7.60442611
Iteration 5, loss = 6.77146826
Iteration 6, loss = 6.11319980
Iteration 7, loss = 5.54456313
Iteration 8, loss = 5.03593371
Iteration 9, loss = 4.61326268
Iteration 10, loss = 4.28923602
Iteration 11, loss = 4.04876052
Iteration 12, loss = 3.87276856
Iteration 13, loss = 3.73917458
Iteration 14, loss = 3.63377422
Iteration 15, loss = 3.54682816
Iteration 16, loss = 3.47243862
Iteration 17, loss = 3.40900235
Iteration 18, loss = 3.35186322
Iteration 19, loss = 3.30136085
Iteration 20, loss = 3.25555113
Iteration 21, loss = 3.21415536
Iteration 22, loss = 3.17561489
Iteration 23, loss = 3.14082761
Iteration 24, loss = 3.10968399
Iteration 25, loss = 3.08118250
Iteration 26, loss = 3.05532372
Iteration 27, loss = 3.03139239
Iteration 28, loss = 3.00960161
Iteration 29, loss = 2.98951386
Iteration 30, loss =

Iteration 252, loss = 2.70292756
Iteration 253, loss = 2.70290360
Iteration 254, loss = 2.70263645
Iteration 255, loss = 2.70249186
Iteration 256, loss = 2.70272301
Iteration 257, loss = 2.70241814
Iteration 258, loss = 2.70216000
Iteration 259, loss = 2.70221586
Iteration 260, loss = 2.70267709
Iteration 261, loss = 2.70194007
Iteration 262, loss = 2.70251234
Iteration 263, loss = 2.70153185
Iteration 264, loss = 2.70157723
Iteration 265, loss = 2.70172150
Iteration 266, loss = 2.70166028
Iteration 267, loss = 2.70174514
Iteration 268, loss = 2.70167440
Iteration 269, loss = 2.70147996
Iteration 270, loss = 2.70107292
Iteration 271, loss = 2.70098506
Iteration 272, loss = 2.70112359
Iteration 273, loss = 2.70110109
Iteration 274, loss = 2.70092641
Iteration 275, loss = 2.70043795
Iteration 276, loss = 2.70090684
Iteration 277, loss = 2.70048909
Iteration 278, loss = 2.70079434
Iteration 279, loss = 2.70045433
Iteration 280, loss = 2.70070054
Iteration 281, loss = 2.70040434
Iteration 

Iteration 152, loss = 2.67834237
Iteration 153, loss = 2.67779073
Iteration 154, loss = 2.67754776
Iteration 155, loss = 2.67732998
Iteration 156, loss = 2.67726712
Iteration 157, loss = 2.67726528
Iteration 158, loss = 2.67707053
Iteration 159, loss = 2.67635952
Iteration 160, loss = 2.67721698
Iteration 161, loss = 2.67644425
Iteration 162, loss = 2.67650766
Iteration 163, loss = 2.67633096
Iteration 164, loss = 2.67568448
Iteration 165, loss = 2.67650630
Iteration 166, loss = 2.67625677
Iteration 167, loss = 2.67579315
Iteration 168, loss = 2.67561686
Iteration 169, loss = 2.67613268
Iteration 170, loss = 2.67537563
Iteration 171, loss = 2.67540953
Iteration 172, loss = 2.67528899
Iteration 173, loss = 2.67487842
Iteration 174, loss = 2.67544275
Iteration 175, loss = 2.67482297
Iteration 176, loss = 2.67505899
Iteration 177, loss = 2.67492130
Iteration 178, loss = 2.67434004
Iteration 179, loss = 2.67427595
Iteration 180, loss = 2.67407490
Iteration 181, loss = 2.67411366
Iteration 

Iteration 401, loss = 2.65831386
Iteration 402, loss = 2.65840335
Iteration 403, loss = 2.65825161
Iteration 404, loss = 2.65813603
Iteration 405, loss = 2.65822878
Iteration 406, loss = 2.65840901
Iteration 407, loss = 2.65799147
Iteration 408, loss = 2.65781918
Iteration 409, loss = 2.65808121
Iteration 410, loss = 2.65777713
Iteration 411, loss = 2.65775170
Iteration 412, loss = 2.65801635
Iteration 413, loss = 2.65825159
Iteration 414, loss = 2.65756662
Iteration 415, loss = 2.65787734
Iteration 416, loss = 2.65754219
Iteration 417, loss = 2.65750539
Iteration 418, loss = 2.65754946
Iteration 419, loss = 2.65735870
Iteration 420, loss = 2.65738196
Iteration 421, loss = 2.65744349
Iteration 422, loss = 2.65751187
Iteration 423, loss = 2.65713035
Iteration 424, loss = 2.65718649
Iteration 425, loss = 2.65753421
Iteration 426, loss = 2.65700488
Iteration 427, loss = 2.65705887
Iteration 428, loss = 2.65654775
Iteration 429, loss = 2.65808478
Iteration 430, loss = 2.65713873
Iteration 

Iteration 203, loss = 2.71899546
Iteration 204, loss = 2.71894993
Iteration 205, loss = 2.71882847
Iteration 206, loss = 2.71866755
Iteration 207, loss = 2.71836415
Iteration 208, loss = 2.71858484
Iteration 209, loss = 2.71811984
Iteration 210, loss = 2.71821771
Iteration 211, loss = 2.71831060
Iteration 212, loss = 2.71781674
Iteration 213, loss = 2.71792564
Iteration 214, loss = 2.71783500
Iteration 215, loss = 2.71745179
Iteration 216, loss = 2.71766385
Iteration 217, loss = 2.71743792
Iteration 218, loss = 2.71768686
Iteration 219, loss = 2.71706901
Iteration 220, loss = 2.71751193
Iteration 221, loss = 2.71733422
Iteration 222, loss = 2.71726360
Iteration 223, loss = 2.71686586
Iteration 224, loss = 2.71659531
Iteration 225, loss = 2.71681933
Iteration 226, loss = 2.71666880
Iteration 227, loss = 2.71655442
Iteration 228, loss = 2.71630598
Iteration 229, loss = 2.71619402
Iteration 230, loss = 2.71638977
Iteration 231, loss = 2.71625162
Iteration 232, loss = 2.71611707
Iteration 

...

[CV 4/5] END regressor__learning_rate=0.1, regressor__loss=lad, regressor__max_depth=5, regressor__n_estimators=200, regressor__random_state=1; MAE: (test=1.514) MSE: (test=5.624) R2: (test=0.647) RMSE: (test=2.372) total time=  13.2s


C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:293: FutureWarning: The loss 'lad' was deprecated in v1.0 and will be removed in version 1.2. Use 'absolute_error' which is equivalent.
  warnings.warn(


[CV 5/5] END regressor__learning_rate=0.1, regressor__loss=lad, regressor__max_depth=5, regressor__n_estimators=200, regressor__random_state=1; MAE: (test=1.473) MSE: (test=5.394) R2: (test=0.638) RMSE: (test=2.322) total time=  14.3s


C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:293: FutureWarning: The loss 'lad' was deprecated in v1.0 and will be removed in version 1.2. Use 'absolute_error' which is equivalent.
  warnings.warn(


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END regressor__max_depth=None, regressor__min_samples_split=2, regressor__n_estimators=100, regressor__random_state=1; MAE: (test=1.646) MSE: (test=6.475) R2: (test=0.596) RMSE: (test=2.545) total time= 1.4min
[CV 2/5] END regressor__max_depth=None, regressor__min_samples_split=2, regressor__n_estimators=100, regressor__random_state=1; MAE: (test=1.685) MSE: (test=6.915) R2: (test=0.567) RMSE: (test=2.630) total time= 1.3min
[CV 3/5] END regressor__max_depth=None, regressor__min_samples_split=2, regressor__n_estimators=100, regressor__random_state=1; MAE: (test=1.649) MSE: (test=6.405) R2: (test=0.603) RMSE: (test=2.531) total time= 1.3min
[CV 4/5] END regressor__max_depth=None, regressor__min_samples_split=2, regressor__n_estimators=100, regressor__random_state=1; MAE: (test=1.673) MSE: (test=6.517) R2: (test=0.591) RMSE: (test=2.553) total time= 1.3min
[CV 5/5] END regressor__max_depth=None, regressor__min_samples_s

In [ ]:
# Imprimir los resultados utilizando la librería tabulate
headers = ['Modelo', 'Mejores Hiperparámetros', 'MAE', 'MSE', 'RMSE', 'R2', 'Tiempo Entrenamiento']
print(tabulate(results, headers=headers, tablefmt='pretty'))

## Modelos de regresión

### Gradient Boosting Regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gbr = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', GradientBoostingRegressor(random_state=0))
    ])

In [ ]:
gbr.fit(X_train, y_train)

In [ ]:
y_pred = gbr.predict(X_val)

In [ ]:
regression_metrics(y_val, y_pred)

## Validación de resultados con el mejor modelo

In [ ]:
from tabulate import tabulate

In [ ]:
y_val_preds = gbr.predict(X_val)

In [ ]:
res = list()
for i in range(len(y_val_preds)):
    y_pred = y_val_preds[i]
    y_real = y_val.iloc[i]
    error = abs(y_pred - y_real)
    res.append([round(y_real, 2), round(y_pred, 2), round(error, 2)])

In [ ]:
print(tabulate(res, headers = ['Real y', 'Predicted y', 'Abs. Error'], tablefmt = 'fancy_grid', numalign = 'center'))

## Simulating data

In [ ]:
line_values = crimes_afluency_ft.line.unique()
type_station_values = crimes_afluency_ft.type.unique()
weekday_values = crimes_afluency_ft.weekday.unique()
crime_category_values = crimes_afluency_ft.id_crime_category.unique()
afluency_dist_values = crimes_afluency_ft.afluency_distribution.unique()

In [ ]:
def random_sample(arr):
    return np.random.choice(arr)

In [ ]:
def simulated_data(unique_values, col_names, n = 50):
    data = [[random_sample(arr) for arr in unique_values] for i in range(n)]
    return pd.DataFrame(data, columns = col_names)

In [ ]:
unique_values = [line_values, type_station_values, weekday_values, crime_category_values, afluency_dist_values]

In [ ]:
features_names = list(crimes_afluency_ft.columns)[:-1]
features_names

In [ ]:
simulated_df = simulated_data(unique_values, features_names, n = 100)

In [ ]:
simulated_df

In [ ]:
y_sim_preds = gbr.predict(simulated_df)

In [ ]:
simulated_df_preds = pd.concat([simulated_df, pd.Series(y_sim_preds, name = 'pred_crime_frequency')], axis = 1)

In [ ]:
simulated_df_preds[(simulated_df_preds['line'] == '2')]

In [ ]:
validation_df_preds = pd.concat([X_val.reset_index(), pd.Series(y_val_preds, name = 'pred_crime_frequency')], axis = 1)

In [ ]:
validation_df_preds[(validation_df_preds['line'] == '2')]

## Export model

In [ ]:
import pickle

In [ ]:
with open('crimes_regressor.pkl', 'wb') as f:
    pickle.dump(gbr, f)